- JSON (JavaScript Object Notation) is a lightweight data format for data exchange
- Using built-in json module for encoding and decoding JSON data
#### Advantages of JSON
- JSON exists as a "sequence of bytes" which is very useful in the case we need to transmit (stream) data over a network. 
- Compared to XML, JSON is much smaller, translating into faster data transfers, and better experiences
- JSON is extremely human-friendly since it is textual, and simultaneously machine-friendly.

Tham khảo: https://docs.python.org/3/library/json.html


#### From Python to JSON (Serialization, Encode)

In [2]:
#Convert Python objects into a JSON string with the json.dumps() method
import json 
person = {"name":"John", "age":30, "city": "NY", "hasChildren": False}

# convert into JSON:
person_json = json.dumps(person)
print(person_json)


{"name": "John", "age": 30, "city": "NY", "hasChildren": false}


In [3]:
# use different formatting  style
person_json2 = json.dumps(person, indent =4, separators = (";","="), sort_keys = True)
print(person_json2)


{
    "age"=30;
    "city"="NY";
    "hasChildren"=false;
    "name"="John"
}


In [4]:
#Convert Python objects into a JSON objects and save them into a file with the json.dump() method
person = {"name":"John", "age":30, "city": "NY", "hasChildren": False}

with open('person.json', 'w') as f:
    json.dump(person, f)

#### From JSON to Python (Deserialization, Decode)

In [6]:
import json 
person_json = """
{
    "age":30,
    "city":"NY",
    "hasChildren":false,
    "name":"John"
}
"""
person = json.loads(person_json)
print(person)

{'age': 30, 'city': 'NY', 'hasChildren': False, 'name': 'John'}


In [7]:
#load datafrom a file and convert it to a Python object with the json.load() method
import json

with open('person.json', 'r') as f:
    person = json.load(f)
    print(person)

{'name': 'John', 'age': 30, 'city': 'NY', 'hasChildren': False}


#### Encoding a custom object


In [8]:
# default JSONEncoder >> raise a TypeError
import json 
def encode_complex(z):
    if isinstance(z, complex):
        return {z.__class__.__name__: True, "real": z.real, "imag":z.imag}
    else:
        raise TypeError(f"Object of type '{z.__class__.__name__}' is not JSON serializable")

z = 5 +9j
zJSON = json.dumps(z, default=encode_complex)
print(zJSON)


{"complex": true, "real": 5.0, "imag": 9.0}


In [9]:
# create a custom Encoder class, and overwrite the default() method
from json import JSONEncoder
class ComplexEncoder(JSONEncoder):
    
    def default(self, o):
        if isinstance(z, complex):
            return {z.__class__.__name__: True, "real":z.real, "imag":z.imag}
        # Let the base class default method handle other objects or raise a TypeError
        return JSONEncoder.default(self, o)
    
z = 5 + 9j
zJSON = json.dumps(z, cls=ComplexEncoder)
print(zJSON)
# or use encoder directly:
zJson = ComplexEncoder().encode(z)
print(zJSON)

{"complex": true, "real": 5.0, "imag": 9.0}
{"complex": true, "real": 5.0, "imag": 9.0}


#### Decoding
- Sử dụng JSONDecoder mặc định, decode thành dictionary


In [10]:
# Possible but decoded as a dictionary
z = json.loads(zJSON)
print(type(z))
print(z)

def decode_complex(dct): # đầu vào là 1 từ điển
    if complex.__name__ in dct:  #tạo đối tượng nếu tìm thấy tên lớp đối tượng trong từ điển
        return complex(dct["real"], dct["imag"])
    return dct

# Now the object is of type complex after decoding
z = json.loads(zJSON, object_hook=decode_complex) #sử dụng hàm này cho đối số object_hook
print(type(z))
print(z)

<class 'dict'>
{'complex': True, 'real': 5.0, 'imag': 9.0}
<class 'complex'>
(5+9j)


In [11]:
#Template encode and decode functions
class User:
    # Custom class with all class variables given in the __init__()
    def __init__(self, name, age, active, balance, friends):
        self.name = name
        self.age = age
        self.active = active
        self.balance = balance
        self.friends = friends
        
class Player:
    # Other custom class
    def __init__(self, name, nickname, level):
        self.name = name
        self.nickname = nickname
        self.level = level
          
            
def encode_obj(obj):
    """
    Takes in a custom object and returns a dictionary representation of the object.
    This dict representation also includes the object's module and class names.
    """
  
    #  Populate the dictionary with object meta data 
    obj_dict = {
      "__class__": obj.__class__.__name__,
      "__module__": obj.__module__
    }
  
    #  Populate the dictionary with object properties
    obj_dict.update(obj.__dict__)
  
    return obj_dict


def decode_dct(dct):
    """
    Takes in a dict and returns a custom object associated with the dict.
    It makes use of the "__module__" and "__class__" metadata in the dictionary
    to know which object type to create.
    """
    if "__class__" in dct:
        # Pop ensures we remove metadata from the dict to leave only the instance arguments
        class_name = dct.pop("__class__")
        
        # Get the module name from the dict and import it
        module_name = dct.pop("__module__")
        
        # We use the built in __import__ function since the module name is not yet known at runtime
        module = __import__(module_name)
        
        # Get the class from the module
        class_ = getattr(module,class_name)

        # Use dictionary unpacking to initialize the object
        # Note: This only works if all __init__() arguments of the class are exactly the dict keys
        obj = class_(**dct)
    else:
        obj = dct
    return obj

# User class works with our encoding and decoding methods
user = User(name = "John",age = 28, friends = ["Jane", "Tom"], balance = 20.70, active = True)

userJSON = json.dumps(user,default=encode_obj,indent=4, sort_keys=True)
print(userJSON)

user_decoded = json.loads(userJSON, object_hook=decode_dct)
print(type(user_decoded))

# Player class also works with our custom encoding and decoding
player = Player('Max', 'max1234', 5)
playerJSON = json.dumps(player,default=encode_obj,indent=4, sort_keys=True)
print(playerJSON)

player_decoded = json.loads(playerJSON, object_hook=decode_dct)
print(type(player_decoded))

{
    "__class__": "User",
    "__module__": "__main__",
    "active": true,
    "age": 28,
    "balance": 20.7,
    "friends": [
        "Jane",
        "Tom"
    ],
    "name": "John"
}
<class '__main__.User'>
{
    "__class__": "Player",
    "__module__": "__main__",
    "level": 5,
    "name": "Max",
    "nickname": "max1234"
}
<class '__main__.Player'>
